Jupyter Notebook used to analyse the average and variance accuracy of the models.

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import json
from collections import Counter
import ast

# Path to the folder containing the models
name_folder = "model_run"
# Number of csv files "model_run{i+1}_simplified.csv"
number_models = 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Function use to calculate the true and false positive probability for every classified label.
def compute_tp_fp(true_labels, predicted_labels):
    true_labels = ast.literal_eval(true_labels)
    predicted_labels = ast.literal_eval(predicted_labels)

    true_counts = Counter(true_labels)
    pred_counts = Counter(predicted_labels)

    tp_counts = Counter(t for t, p in zip(true_labels, predicted_labels) if t == p)
    fp_counts = Counter(p for t, p in zip(true_labels, predicted_labels) if t != p)

    results = []
    for label in true_counts:
        tp_rate = (tp_counts[label] / true_counts[label]) * 100 if true_counts[label] else 0
        fp_rate = (fp_counts[label] / pred_counts[label]) * 100 if pred_counts[label] else 0
        results.append((label, tp_rate, fp_rate))
    
    return results

In [ ]:
# Recovery of the initial identities to recover their number of attributes
name = "../typescript/credentials/identity_attributes_driving_license.csv"
df_labels = pd.read_csv(name)
df_labels["JSON information"] = df_labels["JSON information"].apply(lambda x: json.loads(x))
df_labels["number_attributes"]= 12+df_labels["Number of general additional informations"]+3*df_labels["Number of categories"]+df_labels["Number of additional inforations of each categories"].apply(lambda x: sum(map(int, x.split(','))))

In [ ]:
# Recovery of the average and mean of the scores of the models
score_results = []
for i in range(number_models):
    name_csv = f"model_run{i+1}_simplified.csv"
    df = pd.read_csv(f"{name_folder}/{name_csv}", index_col=0)
    for j, row in df.iterrows():
        # Recovery of the score of each models
        score_results.append({
            "type_model": row["type_model"],
            "regex": row["regex"],
            "score": row["score"],
            "score2": row["score2"],
            "score10": row["score10"]
        })

# Convert to DataFrame
df_score_results =  pd.DataFrame(score_results)

# Group by type_model and regex then compute mean and variance
df_score_results_groupby = df_score_results.groupby(["type_model", "regex"]).agg(
    Score_Mean=("score", "mean"),
    Score_Var=("score", "var"),
    Score2_Mean=("score2", "mean"),
    Score2_Var=("score2", "var"),
    Score10_Mean=("score10", "mean"),
    Score10_Var=("score10", "var")
).reset_index()

# Store the result
df_score_results_groupby.to_csv(f"{name_folder}/df_score_results_groupby.csv")

In [ ]:
# Recovery of the TP and FP percentage and variance of the models for each number of attributes

tp_fp_results = []
score_results = []
for i in range(number_models):
    name_csv = f"model_run{i+1}_simplified.csv"
    df = pd.read_csv(f"{name_folder}/{name_csv}", index_col=0)

    for j, row in df.iterrows():
        # Recovery of the True Positive and False Positive for each label based on its regex and models
        row_results = compute_tp_fp(row["test_labels"], row["predictions"])
        for label, tp_rate, fp_rate in row_results:
            tp_fp_results.append({
                "row": j,
                "type_model": row["type_model"],
                "regex": row["regex"],
                "Personal Number": label,
                "TP%": tp_rate,
                "FP%": fp_rate
            })
        

# Convert to DataFrame
df_tp_fp_results = pd.DataFrame(tp_fp_results).merge(df_labels[["Personal Number", "number_attributes"]], on="Personal Number", how="left")
df_score_results =  pd.DataFrame(score_results)

# Group by type_model, regex, and Personal Number, then compute mean and variance
df_tp_fp_results_groupby = df_tp_fp_results.groupby(["type_model", "regex", "Personal Number"]).agg(
    TP_Mean=("TP%", "mean"),
    TP_Var=("TP%", "var"),
    FP_Mean=("FP%", "mean"),
    FP_Var=("FP%", "var")
).reset_index()

# Group by type_model, regex, and number of attribute, then compute mean and variance
df_tp_fp_results_groupby_number_attribute_only = df_tp_fp_results.groupby(["type_model", "regex", "number_attributes"]).agg(
    TP_Mean=("TP%", "mean"),
    TP_Var=("TP%", "var"),
    FP_Mean=("FP%", "mean"),
    FP_Var=("FP%", "var")
).reset_index()

# Store the data
df_tp_fp_results_groupby_number_attribute_only.to_csv(f"{name_folder}/df_tp_fp_results_groupby_number_attribute_only.csv")
df_tp_fp_results_groupby.to_csv(f"{name_folder}/df_tp_fp_results_groupby.csv")